Setup & Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy as np

path = "/content/drive/MyDrive/data/images/gt_depth/469.png"
gt = cv2.imread(path, cv2.IMREAD_UNCHANGED)

print(gt.dtype, gt.min(), gt.max())


uint8 2 255


In [ ]:
import cv2
import os

# فولدر الصور الناتجة
image_folder = "/content/output_depth"
video_name = "depth_video.mp4"

# اقرأ كل الصور و رتبهم بالاسم
images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
images.sort()  # مهم عشان الترتيب

# اقرأ أول صورة لتحديد أبعاد الفيديو
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

# إعداد الفيديو (30 fps)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # كودك للفيديو
video = cv2.VideoWriter(video_name, fourcc, 10, (width, height))

# أضف كل الصور للفيديو
for img in images:
    frame = cv2.imread(os.path.join(image_folder, img))
    video.write(frame)

video.release()
print(f"✅ تم إنشاء الفيديو: {video_name}")


✅ تم إنشاء الفيديو: depth_video.mp4


In [ ]:
# Zip the specified folder
!zip -r /content/output_depth.zip /content/output_depth

  adding: content/output_depth/ (stored 0%)
  adding: content/output_depth/507_depth.png (deflated 19%)
  adding: content/output_depth/472_depth.png (deflated 16%)
  adding: content/output_depth/510_depth.png (deflated 19%)
  adding: content/output_depth/514_depth.png (deflated 20%)
  adding: content/output_depth/505_depth.png (deflated 20%)
  adding: content/output_depth/511_depth.png (deflated 20%)
  adding: content/output_depth/495_depth.png (deflated 15%)
  adding: content/output_depth/486_depth.png (deflated 26%)
  adding: content/output_depth/483_depth.png (deflated 18%)
  adding: content/output_depth/513_depth.png (deflated 20%)
  adding: content/output_depth/494_depth.png (deflated 15%)
  adding: content/output_depth/471_depth.png (deflated 15%)
  adding: content/output_depth/501_depth.png (deflated 20%)
  adding: content/output_depth/476_depth.png (deflated 14%)
  adding: content/output_depth/508_depth.png (deflated 18%)
  adding: content/output_depth/509_depth.png (deflated 1

In [ ]:
from google.colab import files

# Download the zipped file
files.download('/content/output_depth.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# final code DPT_Hybrid

In [ ]:
import os
import torch
import cv2
import numpy as np
from torchvision.transforms import Compose, Normalize

# 1. اختار الجهاز (GPU لو متاح)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. حمّل موديل MiDaS
midas = torch.hub.load("intel-isl/MiDaS", "DPT_Hybrid")
midas.to(device)
midas.eval()

# 3. حمل الـ transforms الخاصة بالموديل
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
transform = midas_transforms.dpt_transform

# 4. عرف فولدرات
input_folder = "/content/drive/MyDrive/data/images/left"
output_folder = "pre_depth"
os.makedirs(output_folder, exist_ok=True)

# 5. لف على كل الصور في الفولدر
for img_name in os.listdir(input_folder):
    if img_name.lower().endswith((".png", ".jpg", ".jpeg")):
        img_path = os.path.join(input_folder, img_name)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # preprocess
        input_batch = transform(img).to(device)

        with torch.no_grad():
            prediction = midas(input_batch)
            prediction = torch.nn.functional.interpolate(
                prediction.unsqueeze(1),
                size=img.shape[:2],
                mode="bicubic",
                align_corners=False,
            ).squeeze()

        # Convert to numpy
        depth_map = prediction.cpu().numpy()  # real depth values

        # Save depth as 32-bit single channel image
        out_path = os.path.join(output_folder, f"{os.path.splitext(img_name)[0]}.png")
        cv2.imwrite(out_path, depth_map.astype(np.float32))  # preserves real values

        print(f"✅ Processed {img_name} → {out_path}")


Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master
Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


✅ Processed 467.png → pre_depth/467.png
✅ Processed 470.png → pre_depth/470.png
✅ Processed 466.png → pre_depth/466.png
✅ Processed 469.png → pre_depth/469.png
✅ Processed 468.png → pre_depth/468.png
✅ Processed 474.png → pre_depth/474.png
✅ Processed 472.png → pre_depth/472.png
✅ Processed 477.png → pre_depth/477.png
✅ Processed 495.png → pre_depth/495.png
✅ Processed 478.png → pre_depth/478.png
✅ Processed 476.png → pre_depth/476.png
✅ Processed 490.png → pre_depth/490.png
✅ Processed 471.png → pre_depth/471.png
✅ Processed 480.png → pre_depth/480.png
✅ Processed 475.png → pre_depth/475.png
✅ Processed 482.png → pre_depth/482.png
✅ Processed 473.png → pre_depth/473.png
✅ Processed 479.png → pre_depth/479.png
✅ Processed 492.png → pre_depth/492.png
✅ Processed 483.png → pre_depth/483.png
✅ Processed 494.png → pre_depth/494.png
✅ Processed 489.png → pre_depth/489.png
✅ Processed 484.png → pre_depth/484.png
✅ Processed 493.png → pre_depth/493.png
✅ Processed 486.png → pre_depth/486.png


# final code DepthPro-hf

In [ ]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from transformers import DepthProImageProcessorFast, DepthProForDepthEstimation

# 1. الجهاز
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. حمّل الموديل والـ processor
processor = DepthProImageProcessorFast.from_pretrained("apple/DepthPro-hf")
depthpro_model_hf = DepthProForDepthEstimation.from_pretrained("apple/DepthPro-hf").to(device)
depthpro_model_hf.eval()

# 3. الفولدرات
input_folder = "/content/drive/MyDrive/data/images/left"    # حطي هنا مسار فولدر الصور
output_folder = "/content/output_depth"   # فولدر لحفظ النتائج
os.makedirs(output_folder, exist_ok=True)

# 4. المجال بتاع GT
GT_MIN, GT_MAX = 2, 255

# 5. لفة على كل الصور
for file_name in os.listdir(input_folder):
    if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        img_path = os.path.join(input_folder, file_name)
        output_path = os.path.join(output_folder, file_name)  # نفس الاسم جوه فولدر النتائج

        # ---- Load + preprocess ----
        img = Image.open(img_path).convert("RGB")
        inputs = processor(images=img, return_tensors="pt").to(device)

        # ---- استنتاج ----
        with torch.no_grad():
            outputs = depthpro_model_hf(**inputs)

        post_processed = processor.post_process_depth_estimation(
            outputs, target_sizes=[(img.height, img.width)]
        )
        depth_map_m = post_processed[0]["predicted_depth"].cpu().numpy()

        # ---- Normalize [GT_MIN, GT_MAX] ----
        d_min, d_max = depth_map_m.min(), depth_map_m.max()
        depth_scaled = (depth_map_m - d_min) / (d_max - d_min)
        depth_uint8 = (depth_scaled * (GT_MAX - GT_MIN) + GT_MIN).clip(GT_MIN, GT_MAX).astype(np.uint8)

        # ---- Save ----
        cv2.imwrite(output_path, depth_uint8)
        print(f"✅ Processed {img_path} → {output_path}")



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✅ Processed /content/drive/MyDrive/data/images/left/466.png → /content/output_depth/466.png
✅ Processed /content/drive/MyDrive/data/images/left/478.png → /content/output_depth/478.png
✅ Processed /content/drive/MyDrive/data/images/left/467.png → /content/output_depth/467.png
✅ Processed /content/drive/MyDrive/data/images/left/480.png → /content/output_depth/480.png
✅ Processed /content/drive/MyDrive/data/images/left/472.png → /content/output_depth/472.png
✅ Processed /content/drive/MyDrive/data/images/left/476.png → /content/output_depth/476.png
✅ Processed /content/drive/MyDrive/data/images/left/470.png → /content/output_depth/470.png
✅ Processed /content/drive/MyDrive/data/images/left/473.png → /content/output_depth/473.png
✅ Processed /content/drive/MyDrive/data/images/left/481.png → /content/output_depth/481.png
✅ Processed /content/drive/MyDrive/data/images/left/477.png → /content/output_depth/477.png
✅ Processed /content/drive/MyDrive/data/images/left/474.png → /content/output_de